# Data Deep Dive.

In [8]:
!cp /whhdata/dbconnection.json .

In [9]:
%matplotlib inline
import sys
sys.path.insert(0, "..")
import dbutils
from cgmcore import utils
import matplotlib.pyplot as plt
import pandas as pd
import glob2
import os
import config
from tqdm import tqdm
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output
import itertools
from tqdm import tqdm
import glob
import pickle

In [10]:
all_levels = ["scan", "artifact"]
all_datasets = ["all", "training", "nottraining"]
all_poses = ["all", "100", "101", "102", "200", "201", "202"]
all_models = None
limits = [0.2, 0.4, 0.6, 1.2, "all"]

# Getting all models from DB.

In [11]:
NO_MODEL_TOKEN="No model"

# Getting the models.
db_connector = dbutils.connect_to_main_database()
select_sql_statement = "SELECT DISTINCT(type) FROM artifact_quality;"
types = db_connector.execute(select_sql_statement, fetch_all=True)
all_models = sorted([t[0] for t in types if len(t[0]) > 20])[::-1]
print("\n".join(all_models))

20190925-0556_700-175height-10K_samples_frontal
20190914-0703_220-55height
20190913-1807_220-55height
20190806-1551_220-55height
20190723-1119_2550-638weight
20190718-2123_2550-638weight
20190708-0919_2379-595height


# 1. Error on scan and artifact level.

In [12]:
def render_model_details(models):
    
    rows = []
    for model in models:
        search_path = os.path.join("/whhdata/models", model, "*.p")
        path = [path for path in glob.glob(search_path) if path.endswith("-details.p")][0]
        details = pickle.load(open(path, "rb"))

        row = []
        row.append(model)
        row.append(details["epochs"])
        row.append(details["steps_per_epoch"])
        row.append(details["validation_steps"])
        row.append(details["batch_size"])
        row.append(details["dataset_path"].replace("/localssd/preprocessed/", ""))
        row.append(",".join(details["dataset_parameters"]["output_targets"]))
        rows.append(row)

    # Create and render dataframe.
    headers = ("Model", "Epochs", "Steps per Epoch", "Validation Steps", "Batch Size", "Dataset", "Targets")
    df = pd.DataFrame(rows, columns=headers)
    display(df)

In [13]:
error_cache_entries = {}

def error_cache_entry_name(model, level, dataset):
    return "-".join([model, level, dataset])

def update_error_cache_entries(models):
    # Create a cache for the visual results.
    for model, level, dataset in itertools.product(models, all_levels, all_datasets):

        #Cache key.
        cache_key = error_cache_entry_name(model, level, dataset)
        if cache_key in error_cache_entries:
            continue
        
        print("Caching {} {} {}...".format(model, level, dataset))

        # SQL statement for scan level.
        if level == "scan":
            select_sql_statement = ""
            select_sql_statement += "SELECT AVG(aq.value) FROM artifact_quality aq"
            select_sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
            select_sql_statement += " WHERE aq.key='mae'"
            select_sql_statement += " AND aq.type='{}'".format(model)
            if dataset != "all":
                select_sql_statement += " AND misc = '{}'".format(dataset)
            select_sql_statement += " GROUP BY a.qr_code, a.create_date"
            select_sql_statement += ";"

        # SQL statement for artifact level.
        elif level == "artifact":
            select_sql_statement = ""
            select_sql_statement += "SELECT value FROM artifact_quality" 
            select_sql_statement += " WHERE type='{}' AND key='mae'".format(model)
            if dataset != "all":
                select_sql_statement += " AND misc = '{}'".format(dataset)
            select_sql_statement += ";"

        # Retrieve results from DB.
        results = db_connector.execute(select_sql_statement, fetch_all=True)
        results = [x[0] for x in results]

        # Store in cache.
        error_cache_entries[cache_key] = results


In [14]:
def error_visualize(models, level, dataset, bins, maximum_loss, density=False):
        
    update_error_cache_entries(models)
    
    # Just visualize one model, because no second one is selected.
    if len(models) == 1:
    
        # Get results from cache.
        cache_key = error_cache_entry_name(models[0], level, dataset)
        results = error_cache_entries[cache_key]
        results = [result for result in results if result < maximum_loss]

        # Render.
        plt.figure(figsize=(15, 6))
        plt.hist(results, bins=bins, density=density)
        plt.title("Error for {} {}s for {} and model {}.".format(len(results), level, dataset, models[0]))
        plt.axvline(1.2, color="red", linestyle='dashed', linewidth=2, label="Gold Standard")
        plt.legend()
        plt.show()
        plt.close()
    
    # Compare two models.
    else:
        
        # Render.
        plt.figure(figsize=(15, 6))
        
        for model in models:
            cache_key = error_cache_entry_name(model, level, dataset)
            results = error_cache_entries[cache_key]
            results = [result for result in results if result < maximum_loss]
            plt.hist(results, bins=bins, density=density, alpha=0.5 - (len(models) - 1) * 0.05, label=model)
    
        plt.axvline(1.2, color="red", linestyle='dashed', linewidth=2, label="Gold Standard")
        plt.title("Error for {} for {}.".format(level, dataset))
        plt.legend()
        plt.show()
        plt.close()

## Comparing two networks.

In [15]:
# Create interactive widget.
@interact(
    model1=all_models,
    model2=[NO_MODEL_TOKEN] + all_models,
    level=all_levels, 
    dataset=all_datasets, 
    bins=[100, 200, 400, 800],
    maximum_loss=[20, 50, 100, 1000],
    density = [True, False]
)
def interact_visualize(model1, model2, level, dataset, bins, maximum_loss, density):
    
    if model1 == model2:
        print("Both models are the same. Will ignore the second.")
        model2 = NO_MODEL_TOKEN
        
    if model2 == NO_MODEL_TOKEN:
        models = [model1]
    else:
        models = [model1, model2]
        
    error_visualize(models, level, dataset, bins, maximum_loss, density)
    render_model_details(models)
    

interactive(children=(Dropdown(description='model1', options=('20190925-0556_700-175height-10K_samples_frontal…

## Comparing all networks.

In [16]:
# Create interactive widget.
@interact(
    level=all_levels, 
    dataset=all_datasets, 
    bins=[100, 200, 400, 800],
    maximum_loss=[20, 50, 100, 1000],
    density = [True, False]
)
def error_interact_visualize(level, dataset, bins, maximum_loss, density):
    
    models = [model for model in all_models if "height" in model]
    print("Rendering...")
    error_visualize(models, level, dataset, bins, maximum_loss, density)
    render_model_details(models)


interactive(children=(Dropdown(description='level', options=('scan', 'artifact'), value='scan'), Dropdown(desc…

# 2. Gold Standard Results on scan and artifact level.

In [ ]:
goldstandard_cache_entries = {}

def goldstandard_cache_entry_name(model, level, dataset, pose):
    return "-".join([model, level, dataset, pose])

def update_goldstandard_cache_entries(models, poses):
    # Create a cache for the visual results.
    for model, level, dataset, pose in itertools.product(models, all_levels, all_datasets, poses):

        #Cache key.
        cache_key = goldstandard_cache_entry_name(model, level, dataset, pose)
        if cache_key in goldstandard_cache_entries:
            continue
        print("Caching {}...".format(cache_key))


        # Getting scan count from database.
        if level == "scan":
            select_sql_statement = ""
            select_sql_statement += "SELECT COUNT(DISTINCT(m.id))"
            select_sql_statement += " FROM artifact a"
            select_sql_statement += " INNER JOIN measure m ON m.id = a.measure_id"
            select_sql_statement += " INNER JOIN artifact_quality aq ON a.id = aq.artifact_id"
            select_sql_statement += " WHERE aq.key='mae'"
            select_sql_statement += " AND aq.type='{}'".format(model)
            if dataset != "all":
                select_sql_statement += " AND aq.misc = '{}'".format(dataset)
            if pose != "all":
                select_sql_statement += " AND aq.artifact_id LIKE '%_{}_%'".format(pose)
            select_sql_statement += ";"
            all_count = db_connector.execute(select_sql_statement, fetch_one=True)[0]

        # Getting artifact count from database.
        elif level == "artifact":
            select_sql_statement = ""
            select_sql_statement += "SELECT COUNT(*) FROM artifact_quality aq WHERE aq.type='{}' AND aq.key='mae'".format(model)
            if dataset != "all":
                select_sql_statement += " AND aq.misc = '{}'".format(dataset)
            if pose != "all":
                select_sql_statement += " AND aq.artifact_id LIKE '%_{}_%'".format(pose)
            select_sql_statement += ";"
            all_count = db_connector.execute(select_sql_statement, fetch_one=True)[0]

        # Get the rows.
        rows = []
        for limit in limits:

            # SQL statement for scan level.
            if level == "scan":
                select_sql_statement = "SELECT COUNT(v) FROM ("
                select_sql_statement += " SELECT AVG(aq.value) as v FROM artifact_quality aq"
                select_sql_statement += " INNER JOIN artifact a ON a.id = aq.artifact_id"
                select_sql_statement += " INNER JOIN measure m ON m.id = a.measure_id"
                select_sql_statement += " WHERE aq.key='mae'"
                select_sql_statement += " AND aq.type='{}'".format(model)
                if dataset != "all":
                    select_sql_statement += " AND misc = '{}'".format(dataset)
                if pose != "all":
                    select_sql_statement += " AND aq.artifact_id LIKE '%_{}_%'".format(pose)
                select_sql_statement += " GROUP BY m.id"
                select_sql_statement += ") AS subquery"
                if limit != "all":
                    select_sql_statement += " WHERE v <= {}".format(limit)
                select_sql_statement += ";"

            # SQL statement for artifact level.
            elif level == "artifact":
                select_sql_statement = ""
                select_sql_statement += "SELECT COUNT(aq) FROM artifact_quality aq" 
                select_sql_statement += " WHERE type='{}' AND key='mae'".format(model)
                if dataset != "all":
                    select_sql_statement += " AND misc = '{}'".format(dataset)
                if pose != "all":
                    select_sql_statement += " AND aq.artifact_id LIKE '%_{}_%'".format(pose)
                if limit != "all":
                    select_sql_statement += " AND value <={}".format(limit)
                select_sql_statement += ";"

            # Retrieve results from database.
            below_limit_count = db_connector.execute(select_sql_statement, fetch_one=True)[0]

            # Compute per cent.
            if all_count != 0:
                percent = 100 * below_limit_count / all_count
            else:
                percent = 0

            # Come up with a nice description.
            description = "All"
            if limit != "all":
                description = "AE <= {}mm".format(int(10 * limit))

            rows.append((description, below_limit_count, round(percent, 2)))

        # Store in cache.
        goldstandard_cache_entries[cache_key] = rows


In [ ]:
# Create interactive widget.
@interact(model=all_models, level=all_levels, dataset=all_datasets)
def goldstandard_interact_visualize(model, level, dataset):

    # Update the cache.
    update_goldstandard_cache_entries([model], poses=["all"])
    
    # Get data from cache.
    cache_key = goldstandard_cache_entry_name(model, level, dataset, pose="all")
    rows = goldstandard_cache_entries[cache_key]

    # Create and render dataframe.
    headers = ("Description", "Number", "%")
    df = pd.DataFrame(rows, columns=headers)
    display(df)

In [ ]:
# Create interactive widget.
@interact(model=all_models, level=all_levels, dataset=all_datasets, pose=all_poses)
def goldstandard_interact_visualize(model, level, dataset, pose):

    # Update the cache.
    update_goldstandard_cache_entries([model], poses=all_poses)
    
    # Get data from cache.
    cache_key = goldstandard_cache_entry_name(model, level, dataset, pose)
    rows = goldstandard_cache_entries[cache_key]

    # Create and render dataframe.
    headers = ("Description", "Number", "%")
    df = pd.DataFrame(rows, columns=headers)
    display(df)

# Rendering measures selected by high/low error.

In [ ]:
@interact(model=all_models)
def another_interact(model):
    global selected_model 
    selected_model = model

In [ ]:
def get_entries(order):

    select_sql_statement = """
        SELECT * FROM (
            SELECT AVG(aq.value) AS average, m.id AS measure_id, COUNT(aq) AS count
            FROM measure m
            INNER JOIN artifact a ON a.measure_id = m.id
            INNER JOIN artifact_quality aq ON aq.artifact_id = a.id
            WHERE aq.type='{}'
            AND aq.key='mae'
            GROUP BY m.id
        ) AS all_measures
        ORDER BY average {}
        LIMIT 50
        ;
        """.format(selected_model, order)

    results = db_connector.execute(select_sql_statement, fetch_all=True)
    return results

# Graphical user interface.
order_widget = None
error_widget = None
selected_measure_id = None
selecte_model = None

def clear_and_render_widgets():
    clear_output()
    if order_widget != None:
        display(order_widget)
    if error_widget != None:
        display(error_widget)
    if selected_measure_id != None:
        print("Rendering...")
        artifacts, targets = get_artifacts_for_measure(selected_measure_id)
        utils.render_artifacts_as_gallery(artifacts, targets=targets)

def create_order_selector():
    global order_widget
    options = ["ASC", "DESC"]
    order_widget = widgets.Dropdown(
        layout={'width': 'initial'},
        options = options,
        value=options[0],
        description="Order",
    )

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            create_error_selector(change['new'])
            clear_and_render_widgets()
    order_widget.observe(on_change)
    
    create_error_selector(options[0])

    
# Create interactive widget.
#@interact(order=["ASC", "DESC"])
def create_error_selector(order):
    
    results = get_entries(order)
    
    global selected_measure_id
    selected_measure_id = results[0][1]
    
    global error_widget
    options = ["Error: {} Measure: {}".format(error, measure_id) for error, measure_id, _ in results]
    error_widget = widgets.Dropdown(
        layout={'width': 'initial'},
        options = options,
        value=options[0],
        description="Measure",
    )

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            index = options.index(change['new'])
            global selected_measure_id
            selected_measure_id = results[index][1]
            clear_and_render_widgets()
    error_widget.observe(on_change)
    clear_and_render_widgets()
        
def get_artifacts_for_measure(measure_id):
    sql_statement = ""
    sql_statement += "SELECT path, height, weight FROM artifact AS a "
    sql_statement += " INNER JOIN measure m ON m.id = a.measure_id"
    sql_statement += " WHERE a.measure_id='{}'".format(measure_id)
    sql_statement += " AND a.type='rgb'"
    sql_statement += ";"
    paths = db_connector.execute(sql_statement, fetch_all=True)
    
    sql_statement = ""
    sql_statement += "SELECT height, weight FROM measure m "
    sql_statement += " WHERE m.id='{}'".format(measure_id)
    sql_statement += ";"
    targets = db_connector.execute(sql_statement, fetch_one=True)
    
    return paths, targets
  
create_order_selector()

## Available Data in Storage (rgb scans, pcd scans)  => rgb = > 16331 &  pcd => 5049

# TODO Tristan: This can be optimized a lot with glob, I guess.

## Getting the number of GB files 
sum_jpg = 0
sum_pcd = 0

search_path = config.artifacts_path + "/"

block = [".DS_Store", "DEMO_TEST_0001", "._data"]
for file in tqdm((os.listdir(search_path))):
    if(file not in block):
        for file1 in os.listdir(search_path + file):
            if(file1 == "measurements"):
                for file2 in os.listdir(search_path + file + "/" + file1 + "/"):
                    for file3 in os.listdir(search_path + file + "/" + file1 + "/" + file2 + "/"):
                        if(file3=="pc"):
                            sum_pcd  = sum_pcd + len(os.listdir(search_path+file+"/measurements/"+file2+"/"+file3))
                        if(file3=="rgb"):
                            sum_jpg = sum_jpg + len(os.listdir(search_path+file+"/measurements/"+file2+"/"+file3))
print("Number of rgb scans in Storage :  " + str(sum_jpg)) 
print("Number of pc scans in Storage :  " + str(sum_pcd)) 